In [71]:
import numpy as np
from numpy import linalg as LA
from imp import reload
from numpy import array, identity, ones, nonzero, zeros
from numpy import linalg as LA

file = "top250movies.txt"

In [72]:
# Read data from file
try:
    with open(file, encoding="utf-8") as f:
        data = {}
        for line in f:
            line = line.strip().split("/")
            movie = line[0]
            actors = line[1:]
            data[movie] = actors
except FileNotFoundError:
    raise IOError(f"Error: Could not find file {file}.")
except ValueError:
    raise ValueError(f"Error: Invalid data in file {file}.")


In [73]:
def sortPageRank(prdict):
    return sorted(prdict.items(), key = lambda item: item[1], reverse=True)


In [74]:
class pageRank():
    def __init__(self, G, eps, tol, maxiter):
        n = G.shape[0]
        for i in range(n): # normalize columns of G
            ci = LA.norm(G[:,i], 1)
            if ci > 0:
                G[:,i] = G[:,i] / ci
            else: # adjustment for a column of zeros
                G[:,i] = np.ones((n,)) / float(n)
        self.G = G # normalized matrix
        self.eps = eps # probability of jumping to a link on page
        self.size = G.shape[0] # size of matrix
        self.tol = tol # tolerance for power method
        self.maxiter = maxiter # 
        
    def powermethod(self):
        n = self.size
        
        # Get sparse G (as a list)
        
        #list of lists of index of nonzero elements in each row
        nzre = [np.nonzero(self.G[k,:] > 0)[0] for k in range(n)] 
        
        #list of vectors of nonzero elements in each row
        nzv = [self.eps * self.G[k, nzre[k]] for k in range(n)]
        
        oeps = (1.0 - self.eps) / n
        x = np.ones((n,1)) / float(n) # initial vector
        
        xn1 = LA.norm(x, 1)
        ntol = xn1
        niter = 0
        while ntol > self.tol and niter < self.maxiter :
            xold = x       
            for k in range(n):
                x[k] = nzv[k] @ x[nzre[k]] + oeps
                
            xn1  = LA.norm(x, 1)
            x = x / xn1
            ntol = LA.norm(x - xold, 1)
            
            niter += 1
            
        return sortPageRank({k:float(v) for (k,v) in zip(range(n), x)})
    
    def movieRank(self, tourneymovies):
        n = self.size
        x = LA.solve(identity(n)-self.eps*self.G,\
                        (1.0-self.eps)/n*ones((n,1)))
        
        # Sort movies by PageRank
        movie_rankings = sortPageRank({k:float(v) for (k,v) in zip(self.nodes,x)})
        
        t_movies = [movie.strip() for movie in tourneymovies]
        t_rankings = sortPageRank({k:float(x[self.nodes.index(k)]) for k in t_movies})
        
        return movie_rankings, t_rankings


In [76]:
G = ??? # Construct G adjacency matrix
pr = pageRank(G, 0.15, 1e-6, 1000)
movie_rankings = pr.powermethod()
movie_rankings_dict = {str(i): rank for i, rank in enumerate(movie_rankings, 1)}
sorted_movies = sortPageRank(movie_rankings_dict)

for movie, rank in sorted_movies:
    print(f"{movie}: {rank}")


SyntaxError: invalid syntax (2228515201.py, line 1)